In [1]:
import numpy as np
from sklearn import datasets 
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

__1.__  Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [2]:
def standard_scale(X):
    res = (X - X.mean(axis=0)) / X.std(axis=0)
    return res

In [3]:
def accuracy(y, y_pred):
  return np.sum(y == y_pred) / len(y)

In [4]:
def e_metrics(x1, x2):
    distance = np.sum(np.square(x1 - x2))
    return np.sqrt(distance)

In [5]:
def knn(x_train, y_train, x_test, k, weights=False, q=0.5):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответ на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов 
        for d in sorted(test_distances)[0:k]:

            if weights==True:
              # расчитаем вес объекта  
              weight = (q) ** d[0]
              classes[d[1]] += weight
            
            else:
              # подсчитаем частоту появления разных классов
              classes[d[1]] += 1


        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

In [6]:
def pca(X, n_component):

  # Найдем собственные векторы и собственные значения 
  covariance_matrix = X.T @ X
  eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

  # сформируем список кортежей (собственное значение, собственный вектор)
  eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

  # и отсортируем список по убыванию собственных значений
  eig_pairs.sort(key=lambda x: x[0], reverse=True)

  eig_sum = sum(eig_values)
  var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
  cum_var_exp = np.cumsum(var_exp)
  print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}\n')

  # а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
  print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}\n')

  # Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
  W = np.hstack([eig_pairs[i][1].reshape(len(eig_pairs[i][1]),1) for i in range(n_component)])
  print(f'Матрица весов W:\n {W}\n')

  # Сформируем новую матрицу "объекты-признаки"
  Z = X.dot(W)
  return Z

In [7]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X.shape, y.shape

((150, 4), (150,))

In [8]:
# стандартизируем признаки

X_standart = standard_scale(X)

In [9]:
# Разобьем датасет на train и test

X_train, X_test, Y_train, Y_test = train_test_split(X_standart, y,
                                                    test_size=0.3,
                                                    random_state=24,
                                                    stratify=y)
X_train.shape, X_test.shape

((105, 4), (45, 4))

In [10]:
# Получим предсказания для модели KNN без применения PCA
y_pred = knn(X_train, Y_train, X_test, 5, weights=True)

In [11]:
# Применим алгоритм уменьшения размерности PCA

X_pca = pca(X_standart, n_component=2)
X_pca[:10]

Доля дисперсии, описываемая каждой из компонент 
[72.9624454132999, 22.850761786701742, 3.6689218892828737, 0.5178709107154724]

Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]



array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356]])

In [12]:
# Снова разобьем данные

X_train_2, X_test_2, Y_train_2, Y_test_2 = train_test_split(X_pca, y,
                                                            test_size=0.3,
                                                            random_state=24,
                                                            stratify=y)
X_train_2.shape, X_test_2.shape

((105, 2), (45, 2))

In [13]:
# Обучим модель KNN после применения PCA
y_pred_pca = knn(X_train_2, Y_train_2, X_test_2, 5, weights=True)

In [15]:
print(f'Модель KNN до применения PCA, accuracy: {accuracy(Y_test, y_pred):.4f}', 
      f'Модель KNN до применения PCA, accuracy: {accuracy(Y_test_2, y_pred_pca):.4f}', sep='\n')


Модель KNN до применения PCA, accuracy: 0.9778
Модель KNN до применения PCA, accuracy: 0.9111


__2*.__ Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции `numpy.linalg.svd()`

In [16]:
def svd(X, n_features):
  '''Снижает размерность данных c помощью SVD до n_features'''

  # получаем SVD разложение матрицы
  U, S, W = np.linalg.svd(X)

  # создадим матрицу из нулей размера, равного размеру матрицы X
  D = np.zeros(X.shape)

  # запомняем матрицу диагональными элементами
  D[:X.shape[0], :X.shape[0]] = np.diag(S)
  # D[np.diag_indices(min(a.shape))] = S

  # сокращаем матрицы до нужного нам размера
  D = D[:, :n_features]
  W = W[:n_features, :]

  # исходная матрица
  X_base = U.dot(D.dot(W))
  print(f'Base matrix:\n {X_base}\n')

  # снижаем размерность
  X_transform = U.dot(D)
  # X_transform = A.dot(W.T)
  print(f'Compressed matrix:\n {X_transform}\n')

  return X_transform

In [17]:
X = np.array([[1, 2, 3, 4, 5], 
              [5, 6, 7, 8, 9],
              [9, 10, 11, 12, 13]])

X.shape

(3, 5)

In [18]:
svd(X, 2)

Base matrix:
 [[ 1.  2.  3.  4.  5.]
 [ 5.  6.  7.  8.  9.]
 [ 9. 10. 11. 12. 13.]]

Compressed matrix:
 [[ -7.1372378    2.01490362]
 [-15.95949868   0.54258812]
 [-24.78175956  -0.92972739]]



array([[ -7.1372378 ,   2.01490362],
       [-15.95949868,   0.54258812],
       [-24.78175956,  -0.92972739]])

In [19]:
# Проверим, совпадают ли матрицы

pca(X, 2)

Доля дисперсии, описываемая каждой из компонент 
[99.43582362431287, 0.5641763756871232, 3.535940702564946e-15, -5.207539364016948e-18, -3.0580099788874984e-15]

Кумулятивная доля дисперсии по компонентам 
[ 99.43582362 100.         100.         100.         100.        ]

Матрица весов W:
 [[-0.33700462 -0.69744382]
 [-0.38905883 -0.3855298 ]
 [-0.44111304 -0.07361578]
 [-0.49316726  0.23829825]
 [-0.54522147  0.55021227]]



array([[ -7.1372378 ,   2.01490362],
       [-15.95949868,   0.54258812],
       [-24.78175956,  -0.92972739]])